In [6]:
import model as m
import wrangle as w
from importlib import reload

In [2]:
train,validate,test = w.tvt_split(w.wrangle_zillow())

In [7]:
reload(m)
m.select_baseline(train.log_error)

|                 |     SSE |       MSE |     RMSE |
|:----------------|--------:|----------:|---------:|
| Mean Baseline   | 656.398 | 0.0270579 | 0.164493 |
| Median Baseline | 658.419 | 0.0271412 | 0.164746 |
### Because mean outperformed median on all metrics,         we will use mean as our baseline